# Decorator

- 오래 전(PEP-318)에서 함수와 메소드의 기능을 쉽게 수정하기 위한 수단으로 소개됨
- 이전에는 `classmethod`나 `staticmethod`같은 함수가 원래 메소드의 정의를 변형하는데 사용되고 있었음
    - 함수에 변형을 할 때마다 `modifier`함수를 사용해서 함수를 호출한 다음, 같은 이름을 할당해야했음

```python
def origianl(...):
    ...

original = modifier(original)
```

- 위와 같은 방법의 경우 함수를 재할당하는 것을 잊어버리거나 함수 정의가 멀리 떨어져있는 경우에 오류가 발생하기 쉽고 번거롭다
- 앞에 코드는 아래와 같은 방식으로 작성한다.

```python
@modifier
def original(...):
    ...
```

- 데코레이터는 데코레이터 이후에 나오는 것을 데코레이터의 첫 번째 파라미터로 하고, 데코레이터의 결과값을 반환하게하는 **문법적 설탕**
    - 위의 방식과 동일한 표현이지만, 이를 쉽게 표기하는 문법적인 표현이다.  
    
    
- 여기서 `modifier`를 파이썬 용어를 **데코레이터**, `original`을 데코레이팅된(decorated)함수 또는 래핑된(wrapped) 객체라고 한다.
- 원래는 함수와 메소드를 위해서 고안되었지만, 실제로는 어떤 종류의 객체에도 적용이 가능하다.
    - 여기서는 함수, 메소드, 제너레이터, 클래스에 데코레이터를 적용하는 방법까지 살펴본다.
- 주의할 점은 *데코레이터 디자인 패턴*과 혼동하면 안된다.

# 함수 데코레이터

- 데코레이터를 이용해서 함수의 기능을 변경한다.
- 함수에 데코레이터를 적용하는 경우에는 어떤 종류의 로직이라도 적용할 수 있다.  
    
    - 파라미터의 유효성 검사
    - 사전조건 검사
    - 기능 전체를 새롭게 정의
    - 서명 변경
    - 원래 함수의 결과를 캐시
    
<br/>

예를 들어 다음과 같이 도메인의 특정 예외에 대해서 특정 횟수만큼 재시도하는 데코레이터를 만들 수 있다.

```python
@decorator_function_1.py
class ControlledException(Exception):
    """도메인에서 발생하는 일반적인 예외"""
    
def retry(operation):
    @wraps(operation)
    def wrapped(*args, **kwargs):
        last_raised = None
        RETRIES_LIMIT = 3
        for _ in range(RETRIES_LIMIT):
            try:
                return operation(*args, **kwargs)
            except ControlledException as e:
                logger.info("retrying %s", operation.__qualname__)
                last_raised = e
        raise last_raised
        
    return wrapped
```
    
위와같이 `retry` 데코레이터를 만들었다면, 다음과 같이 사용할 수 있다.  

```python
def run_operation(task):
    """실행 중 예외가 발생할 것으로 예상되는 특정 작업을 실행"""
    return task.run()
```

위의 실행 예제에서 `run_operation`위에 있는 `@retry`는 실제로 파이써에서 `run_operation = retry(run_operation)`을 실행하게 해주는 문법적 설탕일 뿐이다.

# 클래스 데코레이터

- 함수에 적용한 것처럼 클래스에도 데코레이터를 사용할 수 있다. (PEP-3129) 유일한 차이점은 데코레이터 함수의 파라미터로 함수가 아닌 클래스를 받는다.
- 클래스 데코레이터의 남용은 부작용을 만들 수 있다. (이는 나중에 살펴보도록 한다)
- 클래스 데코레이터의 장점은 아래와 같다.

    - 코드 재사용과 DRY 원칙의 모든 이점을 공유한다. 
        - 클래스 데코레이터를 사용하면 여러 클래스가 특정 인터페이스나 기준을 따르도록 강제할 수 있다.
        - 여러 클래스에 적용할 검사를 데코레이터에서 한 번만 하면 된다.
        
    - 당장은 작고 간단한 클래스를 생성하고 나중에 데코레이터로 기능을 보강할 수 있다.
    
    - 어떤 클래스에 대해서는 유지보수 시 데코레이터를 사용해 기존 로직을 훨씬 쉽게 변경할 수 있다.
        - 메타클래스와 같은 방법을 사용해 보다 복잡하게 만드는 것은 주로 권자오디지 않는다.
        
<br/>

## 예제.

- 모니터링 플랫폼을 위한 이벤트 시스템은 각 이벤트의 데이터를 변환하여 외부 시스템으로 보내야한다. 그러나 각 이벤트 유형은 데이터 전송 방법에 특별한 점이 있을 수 있다.
- 로그인 이벤트에는 자격 증명과 같은 중요한 정보를 숨겨야한다.
- timestamp와 같은 필드는 특별한 포맷으로 표시하기 때문에 변환이 필요할 수도 있다.

> 위와 같은 경우에 요구 사항을 준수하기 위한 가장 간단한 방법은 각 이벤트마다 직렬화 방법을 정의한 클래스를 만드는 것이다.

```python
class LoginEventSerializer:
    def __init__(self, event):
        self.event = event
        
    def serialize(self) -> dict:
        return {
            "username": self.event.username,
            "password": "**민감한 정보 삭제**",
            "ip": self.event.ip,
            "timestamp": self.event.timestamp.strftime("%Y-%m-%d %H:%M"),
        }
    
class LoginEvent:
    SERIALIZER = LoginEventSerializer
    
    def __init__(self, username, password, ip, timestamp):
        self.username = username
        self.password = password
        self.ip = ip
        self.timestamp = timestamp
        
    def serializer(self) -> dict:
        return self.SERIALIZER(self).serialize()
```
    
위와같은 방식은 처음에는 잘 작동하지만, 시간이 지나면서 시스템을 확장할수록 다음과 같은 문제가 발생하게 된다.

- 클래스가 너무 많아진다.
    - 이벤트 클래스와 직렬화 클래스가 1:1로 맵핑되어 있으므로 직렬화 클래스가 점점 많아지게된다.
    
- 이러한 방법은 충분히 유연하지 않다.
    - 만약 password를 가진 다른 클래스에서도 이 필드를 숨기려면 함수로 분리한 다음 여러 클래스에서 호출해야한다. 이는 코드를 충분히 재사용했다고 볼 수 가 없다.
    
- 표준화
    - serialize() 메소드는 모든 이벤트 클래스에 있어야만 한다. 비록 믹스인을 사용해 다른 클래스로 분리할 수 있지만 상속을 제대로 사용했다고 볼 수 없다.

## 해결책

- 이벤트 인스턴스와 변형 함수를 필터로 받아서 동적으로 객체만들기
    - 필터를 이벤트 인스턴스의 필드들에 적용해 직렬화하는 것

```python
from datetime import datetime

def hide_field(field) -> str:
    return "**민감한 정보 삭제**"

def format_time(field_timestamp: datetime) -> str:
    return field_timestamp.strftime("%Y-%m-%d %H:%M")

def show_origianl(event_field):
    return event_field

class EventSerializer:
    def __init__(self, serialization_fields: dict) -> None:
        self.serialization_fields = serialization_fields
        
    def serialize(self, event) -> dict:
        return {
            field: transformation(getattr(event, field))
            for field, transformation in
            self.serialization_fields.items()
        }
    
class Serialization:
    def __init__(self, **transformations):
        self.serializer = EventSerializer(transformations)
        
    def __call__(self, event_class):
        def serialize_method(event_instance):
            return self.serializer.serialize(event_instance)
        event_class.serialize = serialize_method
        return event_class
    
@Serialization(
    username=show_original,
    password=hide_field,
    ip=show_original,
    timestamp=format_time,
)
class LoginEvent:
    def __init__(self, username, password, ip, timestamp):
        self.username = username
        self.password = password
        self.ip = ip
        self.timestamp = timestamp
```

- 데코레이터를 사용하면, 다른 클래스의 코드를 확인하지 않고도 각 필드가 어떻게 처리되는지 쉽게 할 수 있다.
- 클래스 데코레이터에 전달된 인수를 읽는 것만으로도 `username`, `ip`는 수정되지 않고 `password`는 숨겨지며 `timestamp`는 포매팅된다는 것을 알 수 있다.

<br/>

- 이제 개별 클래스에 serialize() 메소드를 정의하거나 믹스인을 확장할 필요도 없다. 단지 데코레이터만 추가하면 된다.
- 위와 같은 경우가 클래스 데코레이터를 올바르게 사용하는 유일한 예다.

# 다른 유형의 데코레이터

- 이제 데코레이터의 @ 구문이 실제로 무엇을 의미하는지 알았다.
    - 클래스 데코레이터는 callable 객체로 만들고 데코레이터로 사용하면 되는 듯 하다.  
    
- 데코레이터는 단지 함수, 메소드, 클래스에만 적용되지 않는다는 것도 알 수 있다.

- 제너레이터, 코루틴, 데코레이트된 객체에 사용될 수 있다.
    - 즉, 데코레이터는 스택 형태로 쌓을 수 있다.
    
> 기타 내용은 Python3.7 기준이라 생략하였다.

# 데코레이터에 인자 전달

- 이미 데코레이터 그 자체도 훌륭하지만, 파라미터를 전달받아 로직을 추상화할 수 있다면 더 강력해질 수 있다.

- 파라미터를 갖는 데코레이터를 구현하는 방법은 여러가지가 있다.
    - 간접 참조(indirection)을 통해 새로운 레벨의 중첩 함수를 만들기
    - 데코레이터를 위한 클래스 만들기
    
- 위의 경우에 일반적으로 **두번째** 방법이 가독성이 좋다. 
     - 세 단계 이상 중첩된 클로저 함수보다는 객체가 이해하기 쉽기 때문이다.
     
<br/>

## 중첩 함수의 데코레이터

- 거시적으로 데코레이터는 함수를 파라미터로 받아서 함수를 반화하는 함수다.
    - **이런 함수를 고차 함수(higher-order function)이라고 부른다.**
    
- 따라서 데코레이터를 파라미터에 전달하려면 다른 수준의 간접 참조가 필요하다.
    - 첫 번째 함수는 파라미터를 받아서 내부 함수에 전달한다.
    - 두 번째 함수는 데코레이터가 될 함수이다.
    - 세 번째 함수는 데코레이팅의 결과를 반환하는 함수이다.
    
> 즉, 최소 세 단계의 중첩 함수가 필요하다는 뜻이다.

### 첫번째 예제 다시 돌아보기.

- 첫 예제에서 재시도의 횟수는 데코레이터 안에 고정되어있었다.
- 만약에 인스턴스마다 재시도 횟수를 지정하려고 하며, 파라미터에 기본 값도 추가해야한다고 해보자
- 위의 경우에는 함수를 한 단계 더 추가해야한다. 
    - 파라미터에 대한 것
    - 데코레이터 자체에 대한 것.
    
<br/>
데코레이터 코드는 아래와 같은 형태가 된다.

```python
@retry(arg1, arg2, ...)
```
     
    
여기서 `@`구문은 데코레이팅 객체에 대한 연사 결과를 반환하는 것이기 때문에 실제 코드의 의미는 아래와 같다.  
`<original_function> = retry(arg1, arg2, ...)(<original_function>)`

<br/>

원하는 재시도 횟수 외에도 제어하려는 예외 유형(추가 파라미터)를 나타낼 수도 있다.  
이를 반영하면 아래와 같다.  

```python
RETRIES_LIMIT = 3

def with_retry(retries_limit=RETRIES_LIMIT, allowed_exceptions=None):
    allowed_exceptions = allowed_exceptions or (ControlledException,)
    
def retry(operation):
    
    @wraps(operation)
    def wrapped(*args, **kwargs):
        last_raised = None
        for _ in range(retries_limit):
            try:
                return operation(*args, **kwargs)
            except allowed_exceptions as e:
                logger.info("retrying %s due to %s", operation, e)
                last_raised = e
        raise last_raised
        
    return wrapped

return retry
```

<br/>

이제 이를 함수에 적용해보자.

```python
# decorator_parametrized_1.py
@with_retry()
def run_operation(task):
    return task.run()

@with_retry(retries_limit=5)
def run_with_custom_retries_limit(task):
    return task.run()

@with_retry(allowed_exceptions=(AttributeError,))
def run_with_custom_exceptions(task):
    return task.run()

@with retry(retries_limit=4, allowed_exceptions=(ZeroDivisionError, AttributeError))
def run_with_custom_parameters(task):
    return task.run()
```

## 데코레이터 객체

- 앞 예제에서는 세 단계의 중첩된 함수가 필요했다.
- 이를 보다 깔끔하게 구현하기 위해 클래스를 사용하여 데코레이터를 정의할 수 있다.
    - 해당 경우에는 `__init__` 메소드에 파라미터를 전달한 다음 `__call__` 메소드에서 데코레이터의 로직을 구현한다.
    
<br/>

```python
class WithRetry:
    def __init__(self, retries_limit=RETRIES_LIMIT, allowed_exceptions=None):
        self.retries_limit = retries_limit
        self.allowed_exceptions = allowed_exceptions or (ControlledException)
        
    def __call__(self, operation):
        
        @wrap(operation)
        def wrapped(*args, **kwrgs):
            last_raised = None
            
            for _ in range(self.retries_limit):
                try:
                    return operation(*args, **kwargs)
                except self.allowed_exceptions as e:
                    logger.info("retrying %s due to %s", operationm e)
                    last_raised = e
                    
                raise last_raised
                
            return wrapped
```

<br/>

사용 방법은 이전과 거의 유사하다.

```python
@WithRetry(retires_limit=5)
def run_with_custom_retries_limit(task):
    return task.run()
```

# 데코레이터 활용 우수 사례

- 파라미터 변환
- 코드 추적
- 파라미터 유효성 검사
- 재시도 로직 구현
- 일부 반복 작업을 데코레이터로 이동하여 클래스 단순화

In [1]:
print("Hello")

Hello


# 안녕
## 안녕
### 안녕

```python
import torch
```

```bash
> sudo apt-get install pytorch
```

![abc](https://images.pexels.com/photos/414612/pexels-photo-414612.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500)